In [ ]:
from snp_query_box.dsnp_transform import dsnpTransform

In [ ]:
#for dev purpose
import os
USERNAME = os.getenv('USERNAME').lower()
new_pull = True

In [ ]:
#!gcloud auth application-default login

In [ ]:
import os
import pandas as pd
import numpy as np

import datetime as dt
import time
import warnings
from dateutil.relativedelta import relativedelta


from snp_query_box import DsnpHelperFunction, snpQueries
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [ ]:
print(f"User: {USERNAME} Started creating pull_measure_monthly_snapshot_bq in the shared storage")

In [ ]:
start_time = time.time()

In [ ]:
today = dt.datetime.today()
today_str = today.strftime('%Y-%m-%d')
last_date_of_last_month_str = DsnpHelperFunction.last_date_of_last_month(today_str, output_type="string")
last_date_of_last_month = DsnpHelperFunction.last_date_of_last_month(today_str)

last_month = last_date_of_last_month_str[:7]
one_year_rolling_back_date = pd.Timestamp(last_date_of_last_month + dt.timedelta(days=1) - relativedelta(years = 1))
one_year_rolling_back_date_str = one_year_rolling_back_date.strftime('%Y-%m-%d')
print(one_year_rolling_back_date)

first_date_of_year = DsnpHelperFunction.first_date_of_year(today_str, output_type="string")
print(first_date_of_year)
first_date_of_last_month_str = DsnpHelperFunction.first_date_of_last_month(today_str, output_type="string")
print(first_date_of_last_month_str)

pulled_data_date = last_month
comp_report_data_month = last_month
hra_report_data_month = last_month
reporting_end_date = last_date_of_last_month_str
print(reporting_end_date)

In [ ]:
storage_path = r'//mbip/medicarepBI/Projects/COE/DSNP/dsnp_data_storage/monthly_flat_files'
output_path = f'{storage_path}/{reporting_end_date}'

isExist = os.path.exists(output_path)
print(output_path)
if not isExist:
    # Create a new directory because it does not exist
     os.makedirs(output_path)
     print("The new directory is created!")
else:
      print("The folder already exist")
print("Output files will sit in the folder above.")

In [ ]:
#no 2025 data yet
reporting_year1=2025
reporting_year2=2025

In [ ]:
# if transition time first month...
if today_str.split('-')[1] =='01':
    first_date_of_year = str(int(first_date_of_year.split('-')[0])-1) + \
        '-01-01'
    print('Trainsition time, first date of year is updated by previous year:' + first_date_of_year)
    #first_date_of_year = '2024-01-01'

In [ ]:
#locate data dump folder
if new_pull:
    pull_measure_monthly_snapshot = snpQueries.pull_measure_monthly_snapshot_bq(reporting_year1=reporting_year1, reporting_year2=reporting_year2)
    #pull_measure_monthly_snapshot.drop(columns = [ "ingest_time", "filename","process_id", "newreportingdt"], inplace=True)
    pull_measure_monthly_snapshot.to_parquet(f"{output_path}/pull_measure_monthly_snapshot_{str(reporting_year1)}.parquet")
    try:
        pull_measure_monthly_snapshot = pd.read_parquet(f"{output_path}/pull_measure_monthly_snapshot_{str(reporting_year1)}.parquet")

    except:
        pull_measure_monthly_snapshot = snpQueries.pull_measure_monthly_snapshot_bq(reporting_year1=reporting_year1, reporting_year2=reporting_year2)
        #pull_measure_monthly_snapshot.drop(columns = ["ingest_time", "filename","process_id", "newreportingdt"], inplace=True)
        pull_measure_monthly_snapshot.to_parquet(f"{output_path}/pull_measure_monthly_snapshot_{str(reporting_year1)}.parquet")

In [ ]:
#col_amt = [col for col in pull_utilization_12month.columns if col.endswith("paid_amt")]
#pull_utilization_12month[col_amt] = pull_utilization_12month[col_amt].astype(float).round(2)

In [ ]:
print(pull_measure_monthly_snapshot.shape)
print(pull_measure_monthly_snapshot.head(3))

In [ ]:
pull_measure_monthly_snapshot

In [ ]:
print("Process time -- %s seconds" % (time.time() - start_time))